# 🧊 Chapter 3D-5: Depth & The Cube

You have drawn a Triangle. But if you tried to draw two triangles at different depths, you would notice **something wrong**.

### The Problem: Z-Fighting
By default, OpenGL draws objects in the order you call `vao.render()`. If Triangle A is **behind** Triangle B, but you draw A *last*, it will appear **on top** of B.

This is called **Z-Fighting**.

### The Solution: Depth Testing
We enable a feature called the **Depth Buffer**. OpenGL tracks the Z depth of every pixel and only draws if the new pixel is **closer** than the old one.

## 1. Enabling Depth Testing

One line of code fixes everything.

In [ ]:
import pygame
import moderngl

pygame.init()
pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF)
ctx = moderngl.create_context()

# 🔥 THE FIX 🔥
ctx.enable(moderngl.DEPTH_TEST)

# Now OpenGL will use a "Depth Buffer" to track Z distance

## 2. Building a Cube (8 Vertices, 12 Triangles)

A cube has 6 faces. Each face is 2 triangles. That's **12 triangles** total.

We need to define **36 vertices** (3 per triangle × 12 triangles), OR use **indices** to reuse vertices.

In [ ]:
import numpy as np

# Cube vertices (X, Y, Z)
# Front face = Z = +0.5, Back face = Z = -0.5
cube_vertices = np.array([
    # Front Face (Red)
    -0.5, -0.5,  0.5,   1.0, 0.0, 0.0,
     0.5, -0.5,  0.5,   1.0, 0.0, 0.0,
     0.5,  0.5,  0.5,   1.0, 0.0, 0.0,
    -0.5, -0.5,  0.5,   1.0, 0.0, 0.0,
     0.5,  0.5,  0.5,   1.0, 0.0, 0.0,
    -0.5,  0.5,  0.5,   1.0, 0.0, 0.0,
    
    # Back Face (Green)
    -0.5, -0.5, -0.5,   0.0, 1.0, 0.0,
     0.5,  0.5, -0.5,   0.0, 1.0, 0.0,
     0.5, -0.5, -0.5,   0.0, 1.0, 0.0,
    -0.5, -0.5, -0.5,   0.0, 1.0, 0.0,
    -0.5,  0.5, -0.5,   0.0, 1.0, 0.0,
     0.5,  0.5, -0.5,   0.0, 1.0, 0.0,
    
    # Top Face (Blue)
    -0.5,  0.5, -0.5,   0.0, 0.0, 1.0,
    -0.5,  0.5,  0.5,   0.0, 0.0, 1.0,
     0.5,  0.5,  0.5,   0.0, 0.0, 1.0,
    -0.5,  0.5, -0.5,   0.0, 0.0, 1.0,
     0.5,  0.5,  0.5,   0.0, 0.0, 1.0,
     0.5,  0.5, -0.5,   0.0, 0.0, 1.0,
    
    # Bottom Face (Yellow)
    -0.5, -0.5, -0.5,   1.0, 1.0, 0.0,
     0.5, -0.5,  0.5,   1.0, 1.0, 0.0,
    -0.5, -0.5,  0.5,   1.0, 1.0, 0.0,
    -0.5, -0.5, -0.5,   1.0, 1.0, 0.0,
     0.5, -0.5, -0.5,   1.0, 1.0, 0.0,
     0.5, -0.5,  0.5,   1.0, 1.0, 0.0,
    
    # Right Face (Magenta)
     0.5, -0.5, -0.5,   1.0, 0.0, 1.0,
     0.5,  0.5,  0.5,   1.0, 0.0, 1.0,
     0.5, -0.5,  0.5,   1.0, 0.0, 1.0,
     0.5, -0.5, -0.5,   1.0, 0.0, 1.0,
     0.5,  0.5, -0.5,   1.0, 0.0, 1.0,
     0.5,  0.5,  0.5,   1.0, 0.0, 1.0,
    
    # Left Face (Cyan)
    -0.5, -0.5, -0.5,   0.0, 1.0, 1.0,
    -0.5, -0.5,  0.5,   0.0, 1.0, 1.0,
    -0.5,  0.5,  0.5,   0.0, 1.0, 1.0,
    -0.5, -0.5, -0.5,   0.0, 1.0, 1.0,
    -0.5,  0.5,  0.5,   0.0, 1.0, 1.0,
    -0.5,  0.5, -0.5,   0.0, 1.0, 1.0,
], dtype='f4')

# Each vertex now has 6 floats: X, Y, Z, R, G, B

## 3. Updating the Shader

We need to accept color as an input from the vertex data.

In [ ]:
vert_shader = '''
    #version 330
    in vec3 in_vert;
    in vec3 in_color;
    
    uniform mat4 mvp;
    
    out vec3 v_color;
    
    void main() {
        gl_Position = mvp * vec4(in_vert, 1.0);
        v_color = in_color;
    }
'''

frag_shader = '''
    #version 330
    in vec3 v_color;
    out vec4 f_color;
    
    void main() {
        f_color = vec4(v_color, 1.0);
    }
'''

prog = ctx.program(vertex_shader=vert_shader, fragment_shader=frag_shader)
vbo = ctx.buffer(cube_vertices.tobytes())

# Tell OpenGL: first 3 floats = position, next 3 floats = color
vao = ctx.vertex_array(prog, [(vbo, '3f 3f', 'in_vert', 'in_color')])

## 4. The Spinning Cube

Combine this with Chapter 3's matrices and you have a **spinning colored cube**.

In [ ]:
from pyrr import Matrix44
import time

projection = Matrix44.perspective_projection(45.0, 800/600, 0.1, 100.0)
view = Matrix44.create_look_at(eye=[0, 0, 5], target=[0, 0, 0], up=[0, 1, 0])

clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    # Rotate on both X and Y axes
    t = time.time()
    rotation = Matrix44.from_x_rotation(t) * Matrix44.from_y_rotation(t * 0.7)
    mvp = projection * view * rotation
    
    prog['mvp'].write(mvp.astype('f4').tobytes())
    
    ctx.clear(0.1, 0.1, 0.1)
    vao.render()
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()

## 🛠️ Challenge: The Rainbow Pyramid

A pyramid has 5 faces: 1 base (square) + 4 triangular sides.

1.  Define the 5 vertices (4 corners + 1 tip).
2.  Give each face a unique color.
3.  Enable depth testing and render it spinning.

If you can see all sides rendering correctly (no faces cutting through each other), you have mastered 3D geometry. 🏔️